# Generating NPZ File
This script aims to generate all required information to be placed into SCARLET as fits files. Then combine the comeponents into a single NPZ file. The information required by SCARLET includes:
- **Filters** (GRIZY)
- **Image data** for each filter
- **Weight map** of the image data for each filter
- **Catalogue** of sources within the frame directly from DES
- **PSF** of the image data for each filter

In [1]:
# Import libraries
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm

# PSF specific libraries
from photutils.detection import find_peaks
from astropy.table import Table
from astropy.stats import sigma_clipped_stats
from astropy.nddata import NDData
from photutils.psf import extract_stars, EPSFBuilder
from photutils.segmentation import detect_threshold

# Catalogue conversion specific library
from astropy.wcs import WCS

In [2]:
# Enter the DESJ ID of the object
desj_id = input('Please Enter the DESJ ID of the object: ')

# Convert the DESJ ID into the corresponding AGEL ID
agel_id = 'AGEL' + str(int(np.round(float(desj_id[4:15]))))\
    + str(int(np.round(float(desj_id[15:]))))
print('DESJ ID: ' + desj_id)
print('AGEL ID: ' + agel_id)

DESJ ID: DESJ233551.8640-515217.7600
AGEL ID: AGEL233552-515218


In [3]:
# load reference data for shape and conversions
ref = fits.open(agel_id + '/' + desj_id + '_g.fits')
ref_data = ref[0].data
ref_data.shape[0]

228

### Filters
SCARLET is able to import 5 filters and hence why ground based imaging is used. These filters are G, R, I, Z, Y.

In [4]:
# Filters will be an array of strings
filters = np.array(['g','r','i','z','y'])

### Image Data
For each filter, we will combine the image data together into a (5, x_dim, y_dim) shaped array

In [5]:
# Empty image data array with shape (5, x_dim, y_dim)
images = np.zeros((5, ref_data.shape[0], ref_data.shape[1]))

# Create for loop to import image data for all 5 filters
for i in range(5):
    image_data = fits.open(agel_id + '/' + desj_id + '_' + filters[i] + '.fits')
    images[i] = image_data[0].data

### Weight Map
Extracted similarly to the image data, we will combine the weight maps into a (5, x_dim, y_dim) shaped array.

In [6]:
#  Empty weight array with shape (5, x_dim, y_dim)
weights = np.zeros((5, ref_data.shape[0], ref_data.shape[1]))

# Create for loop to import weight data for all 5 filters
for j in range(5):
    weight_data = fits.open(agel_id + '/' + desj_id + '_' + filters[j] + '.fits')
    weights[j] = weight_data[2].data

### Catalogue
List of x and y coordinates directly downloaded from the DES catalogue.

In [7]:
# Import catalogue of objects
catalogue_des = fits.open(agel_id + '/' + agel_id + '_catalogue.fits')
catalogue_deg = catalogue_des[1].data

# Converting RA and DEC into pixel coordinates
wcs = WCS(header=ref[0].header) # function according to image_data (what we are mapping onto)

ra_deg = catalogue_deg["RA"] # list of RA coords
dec_deg = catalogue_deg["DEC"] # list of DEC coords
ra_pix, dec_pix = wcs.all_world2pix(ra_deg, dec_deg, 0) # transformation from degrees to pixels


# Limiting to pixel in the size of image
x_pix = ra_pix[np.where(np.logical_and(np.logical_and(ra_pix>=0, ra_pix<=ref_data.shape[0]), 
                                           np.logical_and(dec_pix>=0, dec_pix<=ref_data.shape[1])))]
y_pix = dec_pix[np.where(np.logical_and(np.logical_and(ra_pix>=0, ra_pix<=ref_data.shape[0]), 
                                           np.logical_and(dec_pix>=0, dec_pix<=ref_data.shape[1])))]

# Generate an array of coordinate tuples
catalogue = np.zeros((x_pix.size, y_pix.size), dtype=[('x_pix', '<f8'), ('y_pix', '<f8')])

for k in range(x_pix.size):
    catalogue[k] = (x_pix[k],y_pix[k])

### PSF
Generate a PSF using the image data for each filter
#### *To Be Automated*

In [8]:
# Empty PSF array
psfs = np.zeros((5, 101, 101)) # known shape

for l in range(5):
    psf_data = fits.open(agel_id + '/' + filters[l] + '_epsf.fits')
    psfs[l] = psf_data[0].data

### Converting Multiple Fits Files into a Single NPZ FIle

In [9]:
# Convert to NPZ
np.savez_compressed(agel_id + '/' + agel_id + '_ALL', filters=filters,
                    images=images, weights=weights, catalogue=catalogue, psfs=psfs)

# Load the NPZ
data = np.load(agel_id + '/' + agel_id + '_ALL.npz')

# Ensure the NPZ contains all information
print(data.files)
#print(data['filters'])
#print(data['images'])
#print(data['weights'])
#print(data['catalogue'])
#print(data['psfs'])

['filters', 'images', 'weights', 'catalogue', 'psfs']
